In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout,SpatialDropout1D
from keras.layers import Input
from keras.layers.embeddings import Embedding
from keras.layers import Flatten, LSTM
from keras.layers.core import Activation, Dropout, Dense
from keras.models import Model
from keras.callbacks import EarlyStopping
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.layers import Layer, MaxPooling1D, Bidirectional,GlobalMaxPool1D
from keras.layers.convolutional import Conv1D
from keras.layers.pooling import GlobalMaxPooling1D
from imblearn.over_sampling import RandomOverSampler




# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
teste = pd.read_csv('../input/nlp-getting-started/test.csv')
treino = pd.read_csv('../input/nlp-getting-started/train.csv')

In [ ]:
treino_bal = pd.DataFrame()
for i in range(2):
    B = treino[treino['target']==i]
    B = pd.concat([B,B])
    treino_bal = pd.concat([treino_bal,B.head(4342)])

In [ ]:
import re
def clean_data(df):
    # remove any html tag
    df["text"] = df["text"].apply(lambda x: re.sub(r'<.*?>', '', x))
    # replace urls with <url> tag
    df["text"] = df["text"].apply(lambda x: re.sub(r'https?://\S+|www\.\S+', '<url>', x))
    # replace user names with <user> tag
    df["text"] = df["text"].apply(lambda x: re.sub(r'@[a-zA-Z0-9_]+', '<user>', x))
    # replace hashtags with <hashtag> tag
    df["text"] = df["text"].apply(lambda x: re.sub(r'#[a-zA-Z0-9_]+', '<hashtag>', x))
    
    # replace noisy words - here it can be improved
    df["text"] = df["text"].apply(lambda x: x.replace("\x89", "").replace("hÛ_", "").replace("ÛÓ", ""))
    # replace the happy emojis with <smile> tag
    df["text"] = df["text"].apply(lambda x: re.sub(r'(:|;)-?(\)|D|d)', "<smile>", x))
    # replace the sad emojis with <smile> tag
    df["text"] = df["text"].apply(lambda x: re.sub(r'(:|;)-?\(+', "<sad>", x))
    
    return df

treino = clean_data(treino)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
rotulos = treino_bal['text']
keyword = treino_bal['keyword']
location = treino_bal['location']

In [ ]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(rotulos)

tok_rot = tokenizer.texts_to_sequences(rotulos)

In [ ]:
maxi = []
for i in range(len(tok_rot)):
    maxi.append(len(tok_rot[i]))


In [ ]:
maxlen = 31
tok_rot = sequence.pad_sequences(tok_rot, padding='post', maxlen=maxlen)

In [ ]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(rotulos)

In [ ]:
vectorizer = TfidfVectorizer(analyzer = 'word')
rotulos_vector = vectorizer.fit_transform(rotulos)

In [ ]:
classes_dumies = pd.get_dummies(treino['target'])

In [ ]:
X.shape

In [ ]:
features = np.hstack([rotulos_vector])

In [ ]:
len(classes_dumies)

In [ ]:
classes = treino_bal['target']

In [ ]:
#Balancear
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
x_features, y_classes = ros.fit_resample(X, classes)

In [ ]:
classes_dumies = pd.get_dummies(classes)

In [ ]:
base_treino, base_teste, classes_treino, classes_teste = train_test_split(X, classes_dumies, test_size = 0.2, random_state = 3)

In [ ]:
base_treino.shape

In [ ]:
classes_treino.shape

In [ ]:
classes_dumies

In [ ]:
modelo_keras = Sequential()
modelo_keras.add(Dense(21637, input_dim = base_teste.shape[1], activation='relu',kernel_initializer = 'random_uniform')) #6
modelo_keras.add(Dropout(0.5))
modelo_keras.add(Dense(21637,activation='relu'))#6
modelo_keras.add(Dropout(0.5))
modelo_keras.add(Dense(units=2, activation='sigmoid')) #'sigmoid'


In [ ]:
modelo_keras.summary()
modelo_keras.compile(loss = 'binary_crossentropy', optimizer = 'rmsprop', metrics = ['accuracy'])

In [ ]:
historico = modelo_keras.fit(base_treino,classes_treino,epochs=5,validation_data=(base_teste,classes_teste))

In [ ]:
rotulos = teste['text']
Y = vectorizer.fit_transform(rotulos)

In [ ]:
modelo_keras.predict(Y)

In [ ]:
loss, accuracy = modelo_keras.evaluate(base_treino, classes_treino, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = modelo_keras.evaluate(base_teste, classes_teste, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

### LSTM

In [ ]:
deep_inputs = Input(shape=(21637,))
embedding_layer = Embedding(200, 100)(deep_inputs)#, trainable=False
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(2, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])


In [ ]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 50000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 250
# This is fixed.
EMBEDDING_DIM = 100

In [ ]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=base_treino.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(2, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
epochs = 5
batch_size = 64

history = model.fit(base_treino, classes_treino, epochs=epochs, batch_size=batch_size,validation_split=0.1,callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


In [ ]:
model.save("teste.h5")


In [ ]:
history = model.fit(base_treino,classes_treino,epochs=3,validation_data=(base_teste,classes_teste))

### Embedding

In [ ]:
embedding_dim = 50
vocab_size = base_treino.shape[0]

model = Sequential()
model.add(Embedding(input_dim=vocab_size,output_dim=embedding_dim,input_length=maxlen))
model.add(Flatten())
model.add(Dense(10, activation='relu'))
model.add(Dense(2, activation='sigmoid'))


In [ ]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(base_treino, classes_treino,epochs=100,verbose=True,validation_data=(base_teste, classes_teste),batch_size=10)

In [ ]:
loss, accuracy = model.evaluate(base_treino, classes_treino, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(base_teste, classes_teste, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

### CNN

In [ ]:
embedding_dim = 100
vocab_size = base_treino.shape[0]

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(128, 5, activation='relu'))
model.add(MaxPooling1D(pool_size=4))
model.add(LSTM(128, dropout=0.4, recurrent_dropout=0.4))
model.add(Dense(2, activation='sigmoid'))
#model.add(Conv1D(64, 5, activation='relu'))
#model.add(Conv1D(32, 5, activation='relu'))
#model.add(GlobalMaxPooling1D())
#model.add(Conv1D(128, 5, activation='relu'))
#model.add(GlobalMaxPooling1D())
#model.add(Flatten())
#model.add(Dense(128, activation='relu'))
#model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
history = model.fit(base_treino, classes_treino,epochs=10,verbose=False,validation_data=(base_teste, classes_teste),batch_size=10)

In [ ]:
loss, accuracy = model.evaluate(base_treino, classes_treino, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(base_teste, classes_teste, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

## LSTM

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = 20000,output_dim = 256,input_length = 13827))
model.add(Bidirectional(LSTM(256,return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dense(150,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2,activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
history = model.fit(base_treino, classes_treino,epochs=10,verbose=False,validation_data=(base_teste, classes_teste),batch_size=10)

In [ ]:
loss, accuracy = model.evaluate(base_treino, classes_treino, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(base_teste, classes_teste, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

In [ ]:
base_treino.shape[0]

In [ ]:
base_treino = sequence.pad_sequences(base_treino, maxlen=base_treino.shape[0])
base_teste = sequence.pad_sequences(base_teste, maxlen=base_teste.shape[0])

In [ ]:
model = Sequential()
model.add(Embedding(max_features,embedding_dims,input_length=maxlen))
model.add(Dropout(0.2))
model.add(Conv1D(filters,kernel_size,padding='valid',activation='relu',strides=1))
model.add(GlobalMaxPooling1D())
model.add(Dense(hidden_dims))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Dense(2))
model.add(Activation('sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])



In [ ]:
model.fit(base_treino, classe_treino,batch_size=batch_size,epochs=epochs,validation_data=(base_teste, classe_teste))

### SVM

In [ ]:
from sklearn import svm

In [ ]:
clf = svm.SVC(kernel='linear')
clf.fit(x_features, y_classes)

In [ ]:
saida =  clf.predict(base_teste)

In [ ]:
from sklearn import metrics

In [ ]:
print("Accuracy:",metrics.accuracy_score(classes_teste, saida))


In [ ]:
teste = clean_data(teste)
teste

In [ ]:
rotulos_teste = teste['text']

In [ ]:
rotulos_test = vectorizer.fit_transform(rotulos_teste)

In [ ]:
rotulos_test.shape

In [ ]:
x_features.shape

In [ ]:
rotulos_vector.shape

In [ ]:
rotulos_vetor

In [ ]:
s_final =  clf.predict(rotulos_test)

### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)

In [ ]:
base_treino.shape

In [ ]:
from sklearn.feature_selection import chi2
import numpy as np

In [ ]:
chi2(base_treino,  classes_treino)

In [ ]:
for Product, category_id in sorted(category_to_id.items()):
  features_chi2 = chi2(base_treino, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print("# '{}':".format(Product))
  print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-N:])))
  print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-N:])))